In [60]:
import datetime
import json
import os
import firebase_admin
from firebase_admin import credentials, firestore
from googletrans import Translator
from faker import Faker

In [26]:
cred = credentials.Certificate(os.getcwd() + "/cogs121-goduck-firebase-adminsdk-f5baa-73bc1269cd.json")
firebase_admin.initialize_app(cred)

In [83]:
translator = Translator()
fake = Faker()

In [85]:
db = firestore.client()
news_content_ref = db.collection('news_content')
news_overview_ref = db.collection('news_overview')

In [29]:
with open(os.getcwd() + "/fake_news.json", 'r') as f:
    data = temp = json.load(f)

In [87]:
def processNews(title, source, text, category):
    news_content_obj = {"title": title}
    news_content_obj["source"] = source
    news_content_obj["category"] = category
    news_content_obj["translated_title_zh"] = translator.translate(title, dest='zh-cn', src='en').text
    news_content_obj["translated_title_es"] = translator.translate(title, dest='es', src='en').text
    news_content_obj["translated_title_hi"] = translator.translate(title, dest='hi', src='en').text

    p_list = text.split("\n")
    news_content_obj["text"] = []
    
    for p in p_list:
        temp = {}
        temp['en'] = p
        temp['zh'] = translator.translate(p, dest='zh-cn', src='en').text
        temp['es'] = translator.translate(p, dest='es', src='en').text
        temp['hi'] = translator.translate(p, dest='hi', src='en').text
        news_content_obj["text"].append(temp)
        
#     news_content_obj['publish_time'] = fake.date_between(start_date='-3d', end_date='today')
    news_content_obj['publish_time'] = fake.past_datetime(start_date="-3d", tzinfo=None)
    
    news_content_obj_ref = news_content_ref.add(news_content_obj)
    news_content_obj_id = news_content_obj_ref[1].get().id
    
    news_overview_obj = {"title": title}
    news_overview_obj['content_id'] = news_content_obj_id
    news_overview_obj["source"] = source 
    news_overview_obj["publish_time"] = news_content_obj['publish_time']
    news_overview_obj["translated_title_zh"] = news_content_obj["translated_title_zh"]
    news_overview_obj["translated_title_es"] = news_content_obj["translated_title_es"]
    news_overview_obj["translated_title_hi"] = news_content_obj["translated_title_hi"]
    news_overview_obj["category"] = news_content_obj["category"]
    
    news_overview_ref.add(news_overview_obj)

In [88]:
processNews(n['title'], n['source'], n['text'], 0)

TypeError: ('Cannot convert to a Firestore Value', datetime.date(2019, 5, 1), 'Invalid type', <class 'datetime.date'>)

{'source': 'https://www.cnn.com/2019/05/01/tech/google-employees-protest-may-day/index.html',
 'translated_title_zh': '谷歌员工正在举行另一场抗议活动',
 'translated_title_es': 'Los empleados de Google están organizando otra protesta.',
 'translated_title_hi': 'Google कर्मचारी एक और विरोध प्रदर्शन कर रहे हैं',
 'text': [{'en': 'London (CNN Business)Protests are taking place at Google offices Wednesday, following allegations of retaliation from the company against employees who participated in walkouts last year.',
   'zh': '伦敦（美国有线电视新闻网商业）周三，谷歌办公室正在举行抗议活动，此前该公司对参与罢工的员工进行了报复。',
   'es': 'Las protestas en Londres (CNN Business) se llevarán a cabo en las oficinas de Google el miércoles, luego de las acusaciones de represalias de la empresa contra los empleados que participaron en las huelgas el año pasado.',
   'hi': 'लंदन (CNN Business) पिछले साल वॉकआउट में भाग लेने वाले कर्मचारियों के खिलाफ कंपनी द्वारा प्रतिशोध के आरोपों के बाद, बुधवार को Google कार्यालयों में विरोध प्रदर्शन हो रहे हैं।'},
  {'en': "L

datetime.date(2019, 5, 3)